大型語言模型（LLM，如BERT、GPT等）處理文本數據的一般流程。這個過程可以細分為以下幾個步驟：

1. **分詞（Tokenization）**：
   - 這一步驟是將文本字符串分解成較小的單位，稱為“詞元”（tokens）。分詞可以根據不同的規則進行，如基於空格和標點符號分割、子詞分割（如Byte-Pair Encoding, BPE）或使用更複雜的基於規則或學習的方法。

2. **詞元到ID的映射（Token to ID mapping）**：
   - 分詞後，每個詞元會被轉換成一個唯一的數字ID。這些ID對應於語言模型的詞彙表中的索引。這一步是必要的，因為計算機模型不能直接處理文本數據，而是處理數字。

3. **嵌入層（Embedding）**：
   - 在將詞元轉換為ID之後，這些ID將被用來在嵌入層中查找每個詞元的向量表示。嵌入層是一個可訓練的參數矩陣，模型通過學習來調整這些向量，以便更好地捕捉詞義和語法關係。
   - 這些嵌入通常包括詞嵌入，可能還包括位置嵌入（指示詞元在句子中的位置）和分段嵌入（特別是在處理多個句子時）。

4. **模型處理（Model Processing）**：
   - 嵌入向量隨後會被送入語言模型的主體，如Transformer結構。這些模型會通過一系列的層處理這些向量，每一層都包括例如自注意力機制和前饋神經網絡等組件。模型通過這種方式來理解和生成文本上下文的表示。

5. **輸出解析（Output Decoding）**：
   - 對於生成任務，如文本續寫，模型的輸出通常是在詞彙表上的概率分布，這些概率表明下一個詞元是什麼。然後可以使用不同的策略（如貪婪搜索、束搜索）從這些概率中選擇詞元來生成文本。
   - 對於分類任務（如情感分析），模型輸出通常是分類標籤的概率。

6. **後處理（Post-processing）**：
   - 在文本生成任務中，輸出的詞元序列通常會經過後處理轉換回可讀的文本。這可能包括去除特殊詞元、調整格式等步驟。

# Tokenizer - Wordlevel

In [1]:
import torch 
import torch.nn as nn
from pathlib import Path
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = {
    'datasource': 'opus_books',
    'lang_src': 'en',
    'lang_tgt': 'it',
    'tokenizer_file': 'tokenizer_{0}.json'
    }

In [3]:
def get_all_sentences(dataset, language):
    for item in dataset:
        yield item['translation'][language]

def get_or_build_tokenizer(config, dataset, language):
    tokenizer_path = Path(config['tokenizer_file'].format(language))
    if not tokenizer_path.exists():
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(dataset, language), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    
    return tokenizer

In [4]:
# It only has the train split, so we divide it overselves
dataset_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, dataset_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, dataset_raw, config['lang_tgt'])

In [5]:
# Encode a sample sentence
encoding = tokenizer_src.encode("I love a cat.")

# Print the tokens
print("Tokens:", encoding.tokens) # 提供了分词后的词元列表
# Print the token IDs
print("Token IDs:", encoding.ids) # 提供了每个词元对应的在分词器词典中的索引ID。

Tokens: ['I', 'love', 'a', 'cat', '.']
Token IDs: [9, 194, 11, 1812, 7]


# Dataset

- https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-it


In [14]:
dataset_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')
dataset_raw['translation'][:5]

[{'en': 'Source: Project Gutenberg',
  'it': 'Source: www.liberliber.it/Audiobook available here'},
 {'en': 'Jane Eyre', 'it': 'Jane Eyre'},
 {'en': 'Charlotte Bronte', 'it': 'Charlotte Brontë'},
 {'en': 'CHAPTER I', 'it': 'PARTE PRIMA'},
 {'en': 'There was no possibility of taking a walk that day.',
  'it': 'I. In quel giorno era impossibile passeggiare.'}]

In [7]:
from dataset import BilingualDataset

In [16]:
# 假設資料集和分詞器已經定義
dataset = BilingualDataset(dataset_raw, tokenizer_src, tokenizer_tgt, 'en', 'it', 4096)
sample = dataset[1]
sample

{'encoder_input': tensor([  2, 298, 845,  ...,   1,   1,   1]),
 'decoder_input': tensor([  2, 269, 932,  ...,   1,   1,   1]),
 'encoder_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]]], dtype=torch.int32),
 'decoder_mask': tensor([[[1, 0, 0,  ..., 0, 0, 0],
          [1, 1, 0,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]], dtype=torch.int32),
 'label': tensor([269, 932,   3,  ...,   1,   1,   1]),
 'src_text': 'Jane Eyre',
 'tgt_text': 'Jane Eyre'}